In [ ]:
# Transfomers - Scenario 31 - 64 Beams - GPS!

# Average Top-1 accuracy 0.07977207977207977
# Average Top-3 accuracy 0.1794871794871795
# Average Top-5 accuracy 0.27635327635327633
# Average Top-7 accuracy 0.37037037037037035
# Average Top-9 accuracy 0.443019943019943
# Average Top-11 accuracy 0.49145299145299143
# Average Top-13 accuracy 0.5754985754985755
# Average Top-15 accuracy 0.6182336182336182

In [1]:
import os
import random
import datetime
import sys
import shutil

import torch
import torch as t
from torch.utils.data import Dataset, DataLoader
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transf
from torchvision import transforms, utils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import ast

In [ ]:
########################################################################
# Create save directory
########################################################################

In [2]:
# year month day 
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds 
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

07-25-2024
16_45


In [3]:
pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime 
print(pwd)
# Determine whether the folder already exists
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\Main_Folder//saved_folder//07-25-2024_16_45


In [4]:
# Copy the training files to the saved directory
shutil.copy('./scenario31_64_pos_beam_train.csv', pwd)
shutil.copy('./scenario31_64_pos_beam_val.csv', pwd)
shutil.copy('./scenario31_64_pos_beam_test.csv', pwd)

'C:\\Users\\Baqer\\Desktop\\V2X_CNN_All\\Scenario31_64-Beams\\Main_Folder//saved_folder//07-25-2024_16_45\\scenario31_64_pos_beam_test.csv'

In [ ]:
# Create folder to save analysis files and checkpoint

In [5]:
save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory) 
        
isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

In [ ]:
########################################################################
# Data Feeding: Create data sample list
########################################################################

In [6]:
class DataFeed(Dataset):
    def __init__(self, root_dir, nat_sort=False, transform=None, init_shuffle=True):
        self.root = root_dir
        self.samples = self.create_samples(self.root, shuffle=init_shuffle, nat_sort=nat_sort)
        self.transform = transform

    def create_samples(self, root, shuffle=False, nat_sort=False):
        f = pd.read_csv(root)
        data_samples = []
        for idx, row in f.iterrows():
            data = list(row.values[1:])
            data_samples.append(data)
        return data_samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        pos_val = sample[:1]
        pos_val = ast.literal_eval(pos_val[0])
        pos_val = np.asarray(pos_val)
        pos_centers = sample[1:2]
        pos_centers = np.asarray(pos_centers)
        return torch.tensor(pos_val, dtype=torch.float32), torch.tensor(pos_centers, dtype=torch.long)

In [ ]:
########################################################################
# Training, Testing, and Validation!
########################################################################

In [ ]:
# Main Model

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import torch.optim as optimizer
import ast

class TransformerModel(nn.Module):
    def __init__(self, num_features, num_output, d_model=512, nhead=8, num_encoder_layers=6):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(num_features, d_model)
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers)
        self.fc = nn.Linear(d_model, num_output)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(0)  # Add batch dimension
        transformer_output = self.transformer(x, x)
        output = self.fc(transformer_output.squeeze(0))
        return output

In [ ]:
# Training

In [8]:
# Training Hyper-parameters
batch_size = 128
val_batch_size = 1
lr = 0.01
decay = 1e-4
num_epochs = 20
train_size = [1]

# Hyperparameters for our network
input_size = 2
node = 512
output_size = 65

In [9]:
train_dir = 'scenario31_64_pos_beam_train.csv'
val_dir = 'scenario31_64_pos_beam_val.csv'

train_loader = DataLoader(DataFeed(train_dir),
                          batch_size=batch_size,
                          shuffle=False)
val_loader = DataLoader(DataFeed(val_dir),
                        batch_size=val_batch_size,
                        shuffle=False)

class TransformerBeamPred(nn.Module):
    def __init__(self, num_features, num_output):
        super(TransformerBeamPred, self).__init__()
        self.transformer = TransformerModel(num_features, num_output)

    def forward(self, inputs):
        return self.transformer(inputs)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = TransformerBeamPred(input_size, num_output=output_size).to(device)
val_acc = []

with torch.cuda.device(0):
    top_1 = np.zeros((1, len(train_size)))
    top_3 = np.zeros((1, len(train_size)))
    top_5 = np.zeros((1, len(train_size)))
    top_7 = np.zeros((1, len(train_size)))
    top_9 = np.zeros((1, len(train_size)))
    top_11 = np.zeros((1, len(train_size)))
    top_13 = np.zeros((1, len(train_size)))
    top_15 = np.zeros((1, len(train_size)))
    acc_loss = 0
    itr = []

    for idx, n in enumerate(train_size):
        print('Training size is {}'.format(n))
        net = model
        layers = list(net.children())
        criterion = nn.CrossEntropyLoss()
        opt = optimizer.Adam(net.parameters(), lr=lr, weight_decay=decay)
        LR_sch = optimizer.lr_scheduler.MultiStepLR(opt, [15, 25, 40], gamma=0.1, last_epoch=-1)

        count = 0
        running_loss = []
        running_top1_acc = []
        running_top3_acc = []
        running_top5_acc = []
        running_top7_acc = []
        running_top9_acc = []
        running_top11_acc = []
        running_top13_acc = []
        running_top15_acc = []
        best_accuracy = 0

        for epoch in range(num_epochs):
            print('Epoch No. ' + str(epoch + 1))
            skipped_batches = 0
            net.train()
            for tr_count, (pos_data, beam_val) in enumerate(train_loader):
                data = pos_data.to(device)
                label = beam_val[:, 0].to(device)
                opt.zero_grad()
                out = net(data)
                loss = criterion(out, label)
                loss.backward()
                opt.step()
                batch_loss = loss.item()
                acc_loss += batch_loss
                count += 1
                if count % 100 == 0:
                    print('Training-Batch No.' + str(count))
                    running_loss.append(batch_loss)
                    itr.append(count)
                    print('Loss = ' + str(running_loss[-1]))

            print('Start Validation')
            ave_top1_acc = 0
            ave_top3_acc = 0
            ave_top5_acc = 0
            ave_top7_acc = 0
            ave_top9_acc = 0
            ave_top11_acc = 0
            ave_top13_acc = 0
            ave_top15_acc = 0

            ind_ten = torch.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')

            top1_pred_out = []
            top3_pred_out = []
            top5_pred_out = []
            top7_pred_out = []
            top9_pred_out = []
            top11_pred_out = []
            top13_pred_out = []
            top15_pred_out = []

            total_count = 0
            gt_beam = []

            # To track validation loss
            val_loss = 0

            net.eval()
            with torch.no_grad():
                for val_count, (pos_data, beam_val) in enumerate(val_loader):
                    data = pos_data.to(device)
                    labels = beam_val[:, 0].to(device)
                    gt_beam.append(labels.detach().cpu().numpy()[0].tolist())
                    total_count += labels.size(0)
                    out = net(data)
                    loss = criterion(out, labels)  # Calculate validation loss
                    val_loss += loss.item()  # Accumulate validation loss
                    _, top_1_pred = torch.max(out, dim=1)
                    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0].tolist())
                    sorted_out = torch.argsort(out, dim=1, descending=True)

                    top_3_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

                    top_5_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

                    top_7_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

                    top_9_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

                    top_11_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

                    top_13_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

                    top_15_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

                    reshaped_labels = labels.reshape((labels.shape[0], 1))
                    tiled_3_labels = reshaped_labels.repeat(1, 3)
                    tiled_5_labels = reshaped_labels.repeat(1, 5)
                    tiled_7_labels = reshaped_labels.repeat(1, 7)
                    tiled_9_labels = reshaped_labels.repeat(1, 9)
                    tiled_11_labels = reshaped_labels.repeat(1, 11)
                    tiled_13_labels = reshaped_labels.repeat(1, 13)
                    tiled_15_labels = reshaped_labels.repeat(1, 15)

                    batch_top1_acc = torch.sum(top_1_pred == labels, dtype=torch.float32)
                    batch_top3_acc = torch.sum(top_3_pred == tiled_3_labels, dtype=torch.float32)
                    batch_top5_acc = torch.sum(top_5_pred == tiled_5_labels, dtype=torch.float32)
                    batch_top7_acc = torch.sum(top_7_pred == tiled_7_labels, dtype=torch.float32)
                    batch_top9_acc = torch.sum(top_9_pred == tiled_9_labels, dtype=torch.float32)
                    batch_top11_acc = torch.sum(top_11_pred == tiled_11_labels, dtype=torch.float32)
                    batch_top13_acc = torch.sum(top_13_pred == tiled_13_labels, dtype=torch.float32)
                    batch_top15_acc = torch.sum(top_15_pred == tiled_15_labels, dtype=torch.float32)

                    ave_top1_acc += batch_top1_acc.item()
                    ave_top3_acc += batch_top3_acc.item()
                    ave_top5_acc += batch_top5_acc.item()
                    ave_top7_acc += batch_top7_acc.item()
                    ave_top9_acc += batch_top9_acc.item()
                    ave_top11_acc += batch_top11_acc.item()
                    ave_top13_acc += batch_top13_acc.item()
                    ave_top15_acc += batch_top15_acc.item()

            print("Total validation examples:", total_count)
            running_top1_acc.append(ave_top1_acc / total_count)
            running_top3_acc.append(ave_top3_acc / total_count)
            running_top5_acc.append(ave_top5_acc / total_count)
            running_top7_acc.append(ave_top7_acc / total_count)
            running_top9_acc.append(ave_top9_acc / total_count)
            running_top11_acc.append(ave_top11_acc / total_count)
            running_top13_acc.append(ave_top13_acc / total_count)
            running_top15_acc.append(ave_top15_acc / total_count)

            print('Validation Loss: {}'.format(val_loss / len(val_loader)))  # Print the validation loss
            print('Average Top-1 accuracy {}'.format(running_top1_acc[-1]))
            print('Average Top-3 accuracy {}'.format(running_top3_acc[-1]))
            print('Average Top-5 accuracy {}'.format(running_top5_acc[-1]))
            print('Average Top-7 accuracy {}'.format(running_top7_acc[-1]))
            print('Average Top-9 accuracy {}'.format(running_top9_acc[-1]))
            print('Average Top-11 accuracy {}'.format(running_top11_acc[-1]))
            print('Average Top-13 accuracy {}'.format(running_top13_acc[-1]))
            print('Average Top-15 accuracy {}'.format(running_top15_acc[-1]))

            cur_accuracy = running_top1_acc[-1]

            print("Current accuracy:", cur_accuracy)
            print("Best accuracy:", best_accuracy)
            if cur_accuracy > best_accuracy:
                print("Saving the best model")
                net_name = checkpoint_directory + '/transformer_beam_pred-64beams'
                torch.save(net.state_dict(), net_name)
                best_accuracy = cur_accuracy
            print("Updated best accuracy:", best_accuracy)

            print("Saving the predicted values in a CSV file")
            file_to_save = f'{save_directory}/topk_pred_beam_val_after_{epoch+1}th_epoch.csv'
            indx = np.arange(1, len(top1_pred_out) + 1, 1)
            df1 = pd.DataFrame()
            df1['index'] = indx
            df1['link_status'] = gt_beam
            df1['top1_pred'] = top1_pred_out
            df1['top3_pred'] = top3_pred_out
            df1['top5_pred'] = top5_pred_out
            df1['top7_pred'] = top7_pred_out
            df1['top9_pred'] = top9_pred_out
            df1['top11_pred'] = top11_pred_out
            df1['top13_pred'] = top13_pred_out
            df1['top15_pred'] = top15_pred_out
            df1.to_csv(file_to_save, index=False)

            LR_sch.step()

        top_1[0, idx] = running_top1_acc[-1]
        top_3[0, idx] = running_top3_acc[-1]
        top_5[0, idx] = running_top5_acc[-1]
        top_7[0, idx] = running_top7_acc[-1]
        top_9[0, idx] = running_top9_acc[-1]
        top_11[0, idx] = running_top11_acc[-1]
        top_13[0, idx] = running_top13_acc[-1]
        top_15[0, idx] = running_top15_acc[-1]

Training size is 1
Epoch No. 1
Start Validation
Total validation examples: 2103
Validation Loss: 3.964011821302413
Average Top-1 accuracy 0.04232049453162149
Average Top-3 accuracy 0.14503090822634332
Average Top-5 accuracy 0.2686638135996196
Average Top-7 accuracy 0.34236804564907275
Average Top-9 accuracy 0.44079885877318115
Average Top-11 accuracy 0.4950071326676177
Average Top-13 accuracy 0.5611031859248692
Average Top-15 accuracy 0.6058012363290537
Current accuracy: 0.04232049453162149
Best accuracy: 0
Saving the best model
Updated best accuracy: 0.04232049453162149
Saving the predicted values in a CSV file
Epoch No. 2
Start Validation
Total validation examples: 2103
Validation Loss: 3.7527275394951225
Average Top-1 accuracy 0.07132667617689016
Average Top-3 accuracy 0.1745126010461246
Average Top-5 accuracy 0.2719923918212078
Average Top-7 accuracy 0.3594864479315264
Average Top-9 accuracy 0.42605801236329055
Average Top-11 accuracy 0.47979077508321444
Average Top-13 accuracy 0.5

In [ ]:
# Testing

In [10]:
# Load the model checkpoint
test_dir = './scenario31_64_pos_beam_test.csv'

# Load the test data
test_data = pd.read_csv(test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr'].tolist()

# Load the model checkpoint and prepare for evaluation
checkpoint_path = f'{checkpoint_directory}/transformer_beam_pred-64beams'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()
net = model.cuda()

In [11]:
test_loader = DataLoader(DataFeed(test_dir),
                          batch_size=batch_size,
                          shuffle=False)

In [12]:
print('Start Testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0

ind_ten = torch.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')

top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []

running_top1_acc = []
running_top3_acc = []
running_top5_acc = []
running_top7_acc = []
running_top9_acc = []
running_top11_acc = []
running_top13_acc = []
running_top15_acc = []
total_count = 0

gt_beam = []

for val_count, (pos_data, beam_val) in enumerate(test_loader):
    net.eval()
    data = pos_data.to(device)
    labels = beam_val[:,0].to(device)
    opt.zero_grad()
    gt_beam.extend(labels.detach().cpu().numpy().tolist())  # Append all labels from the batch
    total_count += labels.size(0)
    out = net(data)
    _, top_1_pred = torch.max(out, dim=1)
    top1_pred_out.extend(top_1_pred.detach().cpu().numpy().tolist())
    sorted_out = torch.argsort(out, dim=1, descending=True)

    top_3_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.extend(top_3_pred.detach().cpu().numpy().tolist())

    top_5_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.extend(top_5_pred.detach().cpu().numpy().tolist())

    top_7_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.extend(top_7_pred.detach().cpu().numpy().tolist())

    top_9_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.extend(top_9_pred.detach().cpu().numpy().tolist())

    top_11_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.extend(top_11_pred.detach().cpu().numpy().tolist())

    top_13_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.extend(top_13_pred.detach().cpu().numpy().tolist())

    top_15_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.extend(top_15_pred.detach().cpu().numpy().tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = torch.sum(top_1_pred == labels, dtype=torch.float32)
    batch_top3_acc = torch.sum(top_3_pred == tiled_3_labels, dtype=torch.float32)
    batch_top5_acc = torch.sum(top_5_pred == tiled_5_labels, dtype=torch.float32)
    batch_top7_acc = torch.sum(top_7_pred == tiled_7_labels, dtype=torch.float32)
    batch_top9_acc = torch.sum(top_9_pred == tiled_9_labels, dtype=torch.float32)
    batch_top11_acc = torch.sum(top_11_pred == tiled_11_labels, dtype=torch.float32)
    batch_top13_acc = torch.sum(top_13_pred == tiled_13_labels, dtype=torch.float32)
    batch_top15_acc = torch.sum(top_15_pred == tiled_15_labels, dtype=torch.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("Total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Testing_size {}--No. of skipped batches {}'.format(total_count, skipped_batches))
print('Average Top-1 accuracy {}'.format(running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format(running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format(running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format(running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format(running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format(running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format(running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format(running_top15_acc[-1]))

# Save the predicted values in a CSV file
file_to_save = f'{save_directory}/best_epoch_eval_Test.csv'

indx = np.arange(1, len(gt_beam) + 1, 1)
# Load the model checkpoint
test_dir = './scenario31_64_pos_beam_test.csv'

# Load the test data
test_data = pd.read_csv(test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr'].tolist()

df2 = pd.DataFrame()
df2['index'] = indx
df2['link_status'] = gt_beam  # Add the link_status column
df2['original_unit1_pwr3'] = pwr_60ghz
df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

print("Saving the predicted value in a csv file")

Start Testing
Total examples are 702
Testing_size 702--No. of skipped batches 0
Average Top-1 accuracy 0.07977207977207977
Average Top-3 accuracy 0.1794871794871795
Average Top-5 accuracy 0.27635327635327633
Average Top-7 accuracy 0.37037037037037035
Average Top-9 accuracy 0.443019943019943
Average Top-11 accuracy 0.49145299145299143
Average Top-13 accuracy 0.5754985754985755
Average Top-15 accuracy 0.6182336182336182
Saving the predicted value in a csv file
